In [1]:
!pip install spacy && python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 2.3MB 14.8MB/s 
     |████████████████████████████████| 1.2MB 29.5MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=c84dbb1b613bc6167d8e2db021e874e21fd7da9965666a5656aa4a1d6c433322
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [3]:
from pathlib import Path
import os, re, io
import pandas as pd
import numpy as np
import requests

from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.metrics.pairwise import cosine_similarity

import random
import pickle

from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch.utils.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
folder="/content/drive/MyDrive/Sem 3-2 Online/Question Answering System/Final/Code/1-5 DistilBERT"
QA_df = pd.read_csv(folder+'/faqir_faq.csv',header=[0],index_col=0)
QA_df.head()

,FaqID,FaqQuestion,FaqAnswer
0,8656,"How to remove tree sap from car? Hi,I have som...",I have had good luck with commercial products ...
1,40807,How do I get a sticker off of a car window (p...,Goo Gone is a fantastic product (available at ...
2,57727,How to get rid of garbage disposal odor?,1. Inspect the black rubber cover that fits in...
3,120493,How to maintain a lawn mower?,"To properly maintain your lawnmover, you'll ne..."
4,62732,How to fix vertical blinds? I can no longer re...,Click on the link and follow the steps.


In [6]:
class TextPreprocessor():
    def __init__(self, data_df, column_name=None):
        self.data_df = data_df  
        if not column_name and type(colum_name) == str:
            raise Exception("column name is mandatory. Make sure type is string format")
        self.column = column_name
        self.convert_lowercase()    
        self.applied_stopword = False
        self.processed_column_name = f"processed_{self.column}"
        
    def convert_lowercase(self):
        self.data_df.fillna('',inplace=True)
        self.data_df = self.data_df.apply(lambda column: column.astype(str).str.lower(), axis=0)    

    def remove_question_no(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'^\d+[.]',' ', row))    
        
    def remove_symbols(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'[^A-Za-z0-9\s]', ' ', row))    

    def process(self):
        self.remove_question_no()
        self.remove_symbols()  
        return self.data_df

In [7]:
text_preprocessor = TextPreprocessor(QA_df.copy(), column_name=["FaqQuestion", "FaqAnswer"])
processed_QA_df = text_preprocessor.process()
processed_QA_df.to_csv(folder+'/processed_QA_df.csv')

In [8]:
processed_QA_df

,FaqID,FaqQuestion,FaqAnswer
0,8656,how to remove tree sap from car hi i have som...,i have had good luck with commercial products ...
1,40807,how do i get a sticker off of a car window p...,goo gone is a fantastic product available at ...
2,57727,how to get rid of garbage disposal odor,inspect the black rubber cover that fits in ...
3,120493,how to maintain a lawn mower,to properly maintain your lawnmover you ll ne...
4,62732,how to fix vertical blinds i can no longer re...,click on the link and follow the steps
...,...,...,...
4308,138807,how do you remove other car paint rubs from yo...,rubbing compound then put some wax on it bug...
4309,110019,how to replace bathroom fan,if you didn t buy the fan yet you have to cons...
4310,127097,how do you change the small down lights used i...,the halogen is made such that the actual lamp ...
4311,65707,how do i know how much air to put in car tires...,wow so many answers so few correct it amaz...


In [9]:
def qA_training_dataset_builder(df):
    qA_training_dataset=pd.DataFrame(columns=['FaqQuestion','Answer','Label'],dtype="object")
    n=len(df)
    num_a_dash=5
    for i in df.index:
        a_dash_list=random.sample(list(range(0,i)) + list(range(i+1, n)), num_a_dash)
        data_0 = {'FaqQuestion':[df['FaqQuestion'][i]]*num_a_dash, 'Answer':list(df.FaqAnswer.iloc[a_dash_list]), 'Label':[0]*num_a_dash} 
        qA_training_dataset=qA_training_dataset.append(pd.DataFrame(data_0), ignore_index=True)

    data_1=pd.DataFrame.from_dict({'FaqQuestion':df['FaqQuestion'].copy(), 'Answer':df['FaqAnswer'].copy()})
    data_1['Label']=1
    qA_training_dataset=qA_training_dataset.append(data_1, ignore_index=True) 
    qA_training_dataset.to_csv(folder+'/qA_training_dataset.csv')
    train_data = []
    for j in qA_training_dataset.index:
      train_data.append(InputExample(texts=[qA_training_dataset['FaqQuestion'][j], qA_training_dataset['Answer'][j]], label=qA_training_dataset['Label'][j]))
    return train_data

In [10]:
train_data=qA_training_dataset_builder(processed_QA_df)

In [11]:
word_embedding_model = models.Transformer('distilbert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

qA_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
train_loss = losses.SoftmaxLoss(model=qA_model, sentence_embedding_dimension=qA_model.get_sentence_embedding_dimension(), num_labels=2)

In [13]:
qA_model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=1,
          evaluation_steps=1000,
          warmup_steps=100)

In [14]:
pickle.dump(qA_model, open(folder+'/qA_model.sav', 'wb'))